In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/sample_submission.csv
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train.zip
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train_sessions.csv
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/site_dic.pkl
/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/test_sessions.csv


In [2]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit,cross_val_score,GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression



In [3]:
def write_to_submission_file(predicted_labels,out_file='/kaggle/working/',target='target',index_label="session_id"):
    predicted_df=pd.DataFrame(predicted_labels,index=np.arange(1,predicted_labels.shape[0]+1),columns=[target])
    predicted_df.to_csv(out_file,index_label=index_label)

In [4]:
train_df=pd.read_csv('/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/train_sessions.csv',index_col='session_id',parse_dates=['time1'])
test_df =pd.read_csv('/kaggle/input/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/test_sessions.csv',index_col='session_id',parse_dates=['time1'])
train_df=train_df.sort_values(by='time1')
train_df.head()



,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [5]:
sites=['site%s' % i for i in range(1,11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt',sep=' ',index=None,header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt',sep=' ',index=None,header=None)

In [6]:
%%time
cv=CountVectorizer(ngram_range=(1,3),max_features=50000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train=cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test=cv.transform(inp_test_file)
X_train.shape,X_test.shape

CPU times: user 12.6 s, sys: 345 ms, total: 13 s
Wall time: 13 s


((253561, 50000), (82797, 50000))

In [7]:
y_train = train_df['target'].astype('int').values

In [8]:
time_split = TimeSeriesSplit(n_splits=10)

In [9]:
time_split

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None)

In [10]:
[(el[0].shape,el[1].shape) for el in time_split.split(X_train)]

[((23051,), (23051,)),
 ((46102,), (23051,)),
 ((69153,), (23051,)),
 ((92204,), (23051,)),
 ((115255,), (23051,)),
 ((138306,), (23051,)),
 ((161357,), (23051,)),
 ((184408,), (23051,)),
 ((207459,), (23051,)),
 ((230510,), (23051,))]

In [11]:
logit = LogisticRegression(C=1,random_state=17,solver='liblinear')

In [12]:
%%time
cv_scores = cross_val_score(logit,X_train,y_train,cv=time_split,scoring='roc_auc',n_jobs=1)

CPU times: user 3min 11s, sys: 2min 22s, total: 5min 34s
Wall time: 1min 26s


In [13]:
cv_scores,cv_scores.mean()

(array([0.83141992, 0.64669747, 0.87991757, 0.96315292, 0.84221721,
        0.87840646, 0.94475893, 0.85321988, 0.92987618, 0.90752752]),
 0.8677194063723477)

In [14]:
logit.fit(X_train,y_train)

LogisticRegression(C=1, random_state=17, solver='liblinear')

In [15]:
logit_test_pred = logit.predict_proba(X_test)[:,1]
write_to_submission_file(logit_test_pred,'sub1.csv')

In [16]:
def add_time_features(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X
    

In [17]:
X_train_new = add_time_features(train_df.fillna(0), X_train)
X_test_new = add_time_features(test_df.fillna(0), X_test)


In [18]:
X_train_new.shape, X_test_new.shape

((253561, 50004), (82797, 50004))

In [19]:
cv_scores = cross_val_score(logit, X_train_new, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=1)

In [20]:
cv_scores, cv_scores.mean()

(array([0.87652191, 0.75122979, 0.93062022, 0.97864183, 0.90399748,
        0.93831404, 0.96249083, 0.92731279, 0.94886405, 0.94043404]),
 0.9158426980964409)

In [21]:
logit.fit(X_train_new, y_train)

LogisticRegression(C=1, random_state=17, solver='liblinear')

In [22]:
logit_test_pred2 = logit.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred2, 'subm2.csv') # 0.93843
logit_test_pred2

array([3.74836252e-05, 3.93857051e-08, 7.21704237e-08, ...,
       1.25674321e-04, 1.84671019e-05, 4.35993587e-07])

In [23]:
np.logspace(-2,2,10)

array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])

In [24]:
c_values = np.logspace(-2,2,10)


logit_grid_searcher = GridSearchCV(estimator=logit , param_grid={'C':c_values},scoring='roc_auc',n_jobs=1,cv=time_split,verbose=1)

In [25]:
%%time
logit_grid_searcher.fit(X_train_new, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


CPU times: user 47min 26s, sys: 35min 34s, total: 1h 23min 1s
Wall time: 21min 14s


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
             estimator=LogisticRegression(C=1, random_state=17,
                                          solver='liblinear'),
             n_jobs=1,
             param_grid={'C': array([1.00000000e-02, 2.78255940e-02, 7.74263683e-02, 2.15443469e-01,
       5.99484250e-01, 1.66810054e+00, 4.64158883e+00, 1.29154967e+01,
       3.59381366e+01, 1.00000000e+02])},
             scoring='roc_auc', verbose=1)

In [26]:
logit_grid_searcher.best_score_,logit_grid_searcher.best_params_

(0.9173772223426584, {'C': 0.21544346900318834})

In [27]:
logit_test_pred3 = logit_grid_searcher.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred3, 'subm3.csv') # 0.94242